In [53]:
from __future__ import division
import re
from pandas_profiling import ProfileReport
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle

### Set Parameters

In [54]:
# This flag will control whether models will be created by partition or for complete data set.
# Partition column is QualityStarCluster
create_models_by_partition = True
lasso_penalty = 0.1
GBT_max_depth = 1
GBT_n_estimators = 500

In [55]:
data_file =  "rand_2018.csv"
table_columns = ['a','b','c','d','e']      #dummy
new_model_columns = ['a','b','c','d','e']  #dummy

partition_column = 'QualityStarCluster'
without_partition_model_name = 'All'

drop_col_for_traintest = ['QualityStarCluster','Medicare_provider_number','Hospital_name','State',
                          'Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)',
                          'holdout_sample','Class_3', 'Class_5', 'Class_6','Class_10','Relative_price_for_outpatient_services']
id_cols_for_prediction_table = drop_col_for_traintest
target_col = 'Class_5'
# Target cloumn can be Class_3, Class_5, Class_6, Class_10

# Output File names
featureimportance_table_file = "featureimportance_table.csv"
predictions_table_file = "predictions_table.csv"

# Model readable list of important variables
important_variables_file = 'important_variables.pkl'

# Output Directories
Output_results_directory = 'QoS_Output_Results/'
pickled_model_directory  = 'QoS_Pickled_Models/'
partitioned_models_directory = 'Partitioned_Models/'
unpartitioned_models_directory = 'Unpartitioned_Models/'


### 2. Call Data Prep Script

In [56]:
#%run ./1_Medxoom_code_data_prep_final.ipynb

### 3. Load data in Pandas dataframe

In [57]:
df = pd.read_csv(data_file)
#df.columns = table_columns
#df = df[new_model_columns]
print(df.shape)

(2950, 310)


In [58]:
df = df.fillna(0)

In [59]:
pd.set_option('display.max_columns', None)
df.head()

,Medicare_provider_number,Hospital_name,State,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",Number_of_outpatient_services,Relative_price_for_outpatient_services,tot_expenses_employee_benefits,tot_expenses_admin_and_general,tot_expenses_housekeeping,tot_expenses_dietary,capasset_bldngsfixtrs_end10only,capasset_moveequip_end10only,capasset_subtotal_end10only,capasset_total_end10only,net_expenses_anc_dvc_chgd_only10,totcosts_devices_chgd,inpat_charges_devices_chgd,inpat_charges_total,outpat_charges_devices_chgd,outpat_charges_total,mdcd_net_revenue_only10,chgs_uninsured_charity_only10,costs_uninsured_charity_only10,uncomp_uninsured_charity_only10,baddebt_hospcmplx_only10,mdcr_baddebt_hospcmplx_only10,mdcr_baddebt_ln2701_only10,nonmdcr_baddebt_hospcmplx_only10,nonmdcr_baddebt_costs_only10,nonmdcr_uncomp_costs_only10,unreimb_uncomp_costs_only10,chgs_charity_patients_only10,costs_charity_patients_only10,uncomp_charity_patients_only10,mdcr_inpat_discharges_HMO_only10,expns_admin_and_general_salary,expns_nursing_admin_salary,expns_inp_adultspeds_salary,expns_anc_oper_room_salary,expns_anc_radiology_diag_salary,expns_anc_lab_salary,expns_anc_resp_therapy_salary,expns_outp_emergency_salary,expns_subtotal_salary,total_salaries,expenses_empeebens_other,expns_admin_and_general_other,expns_housekeeping_other,expns_dietary_other,total_other_expenses,capasset_bldngsfixtrs_beginning,capasset_moveequip_beginning,capasset_subtotal_beginning,capasset_total_beginning,capasset_moveequip_purchases,capasset_subtotal_purchases,capasset_total_purchases,net_expenses_cap_bldgs_fixtures,net_expenses_cap_movable_equip,net_expenses_employee_benefits,net_expenses_admin_and_general,net_expenses_operation_of_plant,net_expenses_laundry_and_linen,net_expenses_housekeeping,net_expenses_dietary,net_expenses_nursing_admin,net_expenses_medical_records,net_expenses_inp_adultspeds,net_expenses_anc_oper_room,net_expenses_anc_radiology_diag,net_expenses_anc_lab,net_expenses_anc_resp_therapy,net_expenses_anc_phys_therapy,net_expenses_anc_medsupps_chgd,net_expenses_anc_drgs_chgd,net_expenses_outp_emergency,net_expenses_subtotal,net_expenses_total,totcosts_adultspeds,totcosts_oper_room,totcosts_radiology_diag,totcosts_lab,totcosts_resp_therapy,totcosts_phys_therapy,totcosts_medsupps_chgd,totcosts_drgs_chgd,totcosts_emergency,totcosts_subtotal,totcosts_total,inpat_charges_adultspeds,inpat_charges_oper_room,inpat_charges_radiology_diag,inpat_charges_lab,inpat_charges_resp_therapy,inpat_charges_phys_therapy,inpat_charges_medsupps_chgd,inpat_charges_drgs_chgd,inpat_charges_emergency,inpat_charges_subtotal,outpat_charges_oper_room,outpat_charges_radiology_diag,outpat_charges_lab,outpat_charges_resp_therapy,outpat_charges_phys_therapy,outpat_charges_medsupps_chgd,outpat_charges_drgs_chgd,outpat_charges_emergency,outpat_charges_obs_beds,outpat_charges_subtotal,mdcr_outpat_charges,mdcr_outpat_costs,mdcr_inpat_costs,mdcr_inpat_charges_adpeds,mdcr_inpat_charges_anc_outp_oth,mdcr_outpat_lesser_costchg,gross_patient_rev_inpat_hosp,gross_patient_rev_inpat_anc,gross_patient_rev_inpat,gross_patient_rev_outpat,gross_patient_rev,allowances_discounts,net_patient_rev,operating_expenses,net_income_srvcs_patients,other_income_other,other_income_all,net_income_srvcs_patients_other,net_income,mdcd_charges,mdcd_costs,bed_days_available,mdcr_hmo_inpat_days,mdcr_inpat_days,mdcd_inpat_days,all_inpat_days,mdcr_inpat_discharges_ln1,mdcr_inpat_discharges,mdcd_inpat_discharges_ln1,mdcd_inpat_discharges,all_inpat_discharges_ln1,all_inpat_discharges,wt_cy,n_hosp_years_reporting,source_2552_10,cash_on_hand_and_in_banks,accounts_receivable,inventory,tot_cur_assets_genfund,tot_fixed_assets_genfund,tot_other_assets_genfund,tot_assets_genfund,tot_cur_liab_genfund,tot_longterm_liab_genfund,tot_liab_genfund,tot_fundbal_genfund,tot_liab_fundbal_genfund,cost_to_charge_ratio_for_uncomp,adult_peds_beds,tot_adult_peds_beds,beds,beds_incl_subprov,employees_FTEs,type_hosp,ownership_nonprofit,ownersh

In [60]:
df.dtypes

Medicare_provider_number                                             int64
Hospital_name                                                       object
State                                                               object
Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)    float64
Number_of_outpatient_services                                      float64
                                                                    ...   
Class_3                                                              int64
Class_5                                                              int64
Class_6                                                              int64
Class_10                                                             int64
holdout_sample                                                       int64
Length: 310, dtype: object

### 4. Split data into dictionary of dataframes by region

In [61]:
#Dictionary of dataframes by region
region_dataframes = {}
Region_list = []

if (create_models_by_partition == False):
    Region_list = [without_partition_model_name]
    region_dataframes[without_partition_model_name] = df

else:
    Region_list = df[partition_column].unique().tolist()
    for region in Region_list:
        region_dataframes[region] = df[df[partition_column]==region]

### 5. Data Preparation Functions

In [62]:
def data_prep_region(df_region):
    
    return df_region

In [63]:
def drop_columns_for_modeling_train(x_train):
    x_train = x_train.drop(drop_col_for_traintest, axis =1)
    return x_train

def drop_columns_for_modeling_test(x_test):
    x_test = x_test.drop(drop_col_for_traintest, axis =1)
    return x_test

In [64]:
#train test split
def train_test(df_region):
    Y=df_region[target_col]    
    x_train, x_test, y_train, y_test = train_test_split(df_region,Y,test_size=0.2, random_state=30, stratify=Y)
    return x_train, x_test, y_train, y_test

### 6. Variable Reduction Functions (Decision Tree and Lasso)

In [65]:
#variable reduction using Lasso
def lasso_variable_reduction(x_train, y_train):
    x_train = drop_columns_for_modeling_train(x_train)
    scaler = StandardScaler()
    scaler.fit(x_train.fillna(0))
    sel_ = SelectFromModel(LogisticRegression(C=lasso_penalty, penalty='l1', solver='saga', multi_class='multinomial', random_state = 20))
    sel_.fit(scaler.transform(x_train.fillna(0)), y_train)
    sel_.get_support()
    selected_feat = x_train.columns[(sel_.get_support())]
    
    #print((sel_.estimator_.coef_ != 0).ravel().tolist())
    #selected_features_lasso = x_train.columns[(sel_.estimator_.coef_ != 0).ravel().tolist()]
    
    
    all_variable_coef = dict(zip( x_train.columns.tolist(),sel_.estimator_.coef_[0]))
    
    selected_l_dict = { key:value for (key,value) in all_variable_coef.items() if value != 0}
    return selected_l_dict

### 7. Model Training and Prediction (GBT)

In [66]:
#Model Training GBT
def GBT(x_train, x_test, y_train, y_test, imp_variables):
    x_train = drop_columns_for_modeling_train(x_train)
    x_test = drop_columns_for_modeling_test(x_test)
    red_x_train = x_train[imp_variables]
    red_x_test  = x_test[imp_variables]
    print(imp_variables)
    GB = GradientBoostingClassifier(max_depth = GBT_max_depth, n_estimators=GBT_n_estimators, max_features = 'sqrt',random_state=20)  
    GB.fit(red_x_train,y_train)    
    predictions_train = GB.predict(red_x_train)   #[:,1]
    predictions_test = GB.predict(red_x_test)     #[:,1]
    selected_features_gbt_dict = dict(zip(red_x_train.columns, GB.feature_importances_))
    selected_gbt_features = { key:value for (key,value) in selected_features_gbt_dict.items() if value != 0}
    return predictions_train, predictions_test, selected_features_gbt_dict, GB

### 8. Save Output Tables (Prediction and Feature Importance)

In [67]:
def create_prediction_table(x_data, prediction_probability, traintest_flag, modelname):
    prediction_tab  = pd.DataFrame()
    prediction_tab['predict'] = prediction_probability
    prediction_tab.reset_index(drop=True)
    prediction_tab['TrainTest'] = traintest_flag
    prediction_tab['Model'] = modelname
    prediction_table = pd.concat([x_data[id_cols_for_prediction_table].reset_index(drop=True),prediction_tab.reset_index(drop=True)], axis=1)
    return prediction_table

In [68]:
def create_featureimportance_table(features, modelname, region_name):
    feature_tab = pd.DataFrame.from_dict(features, orient='index')
    feature_tab['Feature'] = feature_tab.index
    feature_tab.reset_index(drop=True,inplace=True)
    feature_tab['Importance'] = feature_tab[[0]]
    feature_tab.drop(0, axis =1, inplace = True)
    feature_tab['Model'] = modelname
    feature_tab[partition_column] = region_name
    return feature_tab

### 9. Save Pickled Models

In [69]:
def save_model(region,model):
    
    model_name = ''
    
    if (create_models_by_partition == True):
        model_name = pickled_model_directory+partitioned_models_directory+region+'_model.pkl'
    else:
        model_name = pickled_model_directory+unpartitioned_models_directory+region+'_model.pkl'
    
    #model_name = pickled_model_directory+region+'_model.pkl'
    with open(model_name,'wb') as fout:
        pickle.dump(model,fout)
    print(model_name)

In [70]:
def save_important_variables(important_variables):
    file_name = ''
    
    if (create_models_by_partition == True):
        file_name = pickled_model_directory+partitioned_models_directory+important_variables_file
    else:
        file_name = pickled_model_directory+unpartitioned_models_directory+important_variables_file
    
    with open(file_name,'wb') as fout:
        pickle.dump(important_variables,fout)

### 10. Initialize Data Structures

In [71]:
#initialize data structures
x_train = {}
x_test = {}
y_train = {}
y_test = {}
lasso_important_variables = {}
dt_important_variables = {}
gbt_important_variables = {}

predictions_train = {}
predictions_test = {}
predictions_table = pd.DataFrame()       #Final table with predictions
featureimportance_table = pd.DataFrame() #Final table with feature importance

### 11. Main function to run the Pipeline

In [72]:
for region in Region_list:
    print("\n\n")
    print(partition_column," = ", region)
    region_dataframes[region] = data_prep_region(region_dataframes[region])
    x_train[region], x_test[region], y_train[region], y_test[region] = train_test(region_dataframes[region])
    
    lasso_important_variables[region] = lasso_variable_reduction(x_train[region], y_train[region].values.ravel())
    print("\n")
    print("Lasso Reduced Variables", len(lasso_important_variables[region]))
    print(lasso_important_variables[region])
       
    predictions_train[region], predictions_test[region], gbt_important_variables[region], model  = GBT(x_train[region], x_test[region], y_train[region].values.ravel(), y_test[region].values.ravel(), sorted(lasso_important_variables[region].keys()))
    
    predictions_table = predictions_table.append(create_prediction_table(x_train[region], predictions_train[region], "train", "GBT"))
    predictions_table = predictions_table.append(create_prediction_table(x_test[region], predictions_test[region], "test", "GBT"))
    
    featureimportance_table = featureimportance_table.append(create_featureimportance_table(gbt_important_variables[region], 'GBT',region))
    featureimportance_table = featureimportance_table.append(create_featureimportance_table(lasso_important_variables[region], 'Lasso',region))
     
    save_model(region,model)
    
    




QualityStarCluster  =  1Star


C:\Users\haseeb.inayat\Documents\Python\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




Lasso Reduced Variables 34
{'tot_expenses_admin_and_general': -0.005155382911989753, 'outpat_charges_devices_chgd': 0.021007206927462187, 'unreimb_uncomp_costs_only10': -0.005021350549955747, 'expns_admin_and_general_other': -0.03151735154536486, 'net_expenses_anc_radiology_diag': -0.009512392936093982, 'totcosts_radiology_diag': -0.027021929658605413, 'totcosts_phys_therapy': -0.020375129101814885, 'totcosts_drgs_chgd': -0.007877880216239367, 'cost_to_charge_ratio_for_uncomp': 0.32625358354116374, 'type_hosp': -0.004073991245950051, 'ownership_forprofit': 0.007066007245726626, 'chow_cnt': -0.0597971168433246, 'mdcr_outpat_cost_to_charge_ratio': 0.37199647225661847, 'mdcr_inpat_costs_per_day': 0.035454096219755885, 'income_cont_invest_approp': -0.0008704761863366716, 'mdcr_outpat_margin': 0.012228800439025841, 'mdcr_margin': 0.17823928906130665, 'mdcr_inpat_day_share': -0.061418837673065094, 'outpat_totcosts_radiology_diag': -0.06055675862470908, 'outpat_totcosts_phys_therapy': -0.03

C:\Users\haseeb.inayat\Documents\Python\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




Lasso Reduced Variables 49
{'baddebt_hospcmplx_only10': -0.022642313981065202, 'nonmdcr_baddebt_hospcmplx_only10': -0.027788196864181925, 'nonmdcr_baddebt_costs_only10': -0.03214425190534158, 'nonmdcr_uncomp_costs_only10': -0.02539333215407444, 'unreimb_uncomp_costs_only10': -0.027034049872204646, 'costs_charity_patients_only10': -0.010979744935639666, 'uncomp_charity_patients_only10': -0.011124031943096635, 'totcosts_radiology_diag': -0.0010990228294628177, 'cost_to_charge_ratio_for_uncomp': 0.17719899505218326, 'ownership_nonprofit': -0.04360115484616565, 'ownership_forprofit': 0.028862817687612716, 'chow_cnt': -0.018896141031109377, 'cbsa_ind': -0.04870195345617415, 'days_in_gross_accounts_recvable': -0.003399559436478095, 'equity_financing': -0.023895274636729413, 'mdcr_inpat_cost_to_charge_ratio': -0.00794043638557236, 'mdcr_inpat_costs_per_day': 0.032483664876728234, 'occupancy': -0.044631551343727684, 'mdcr_outpat_margin': 0.0589192047401029, 'mdcr_margin': 0.0872621686232494,

C:\Users\haseeb.inayat\Documents\Python\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




Lasso Reduced Variables 39
{'baddebt_hospcmplx_only10': -0.029751974004092106, 'nonmdcr_baddebt_hospcmplx_only10': -0.011590538470120785, 'net_expenses_dietary': -0.004071318856730182, 'inpat_charges_lab': -0.005189950017801732, 'mdcr_outpat_charges': -0.011205541318327626, 'mdcr_inpat_charges_anc_outp_oth': -0.00568920594656817, 'mdcr_outpat_lesser_costchg': 0.02963225974662335, 'accounts_receivable': -0.002419068734417145, 'cost_to_charge_ratio_for_uncomp': 0.03639152126885579, 'type_hosp': 0.06707140631355227, 'ownership_nonprofit': -0.009868245353410266, 'ownership_forprofit': 0.05729275610801392, 'cbsa_ind': -0.07595306433543357, 'current_ratio': -0.06995308131339373, 'equity_financing': -0.07099736555431307, 'mdcr_inpat_costs_per_day': 0.013645707460172518, 'occupancy': -0.04511343767147357, 'operating_margin': -0.014847172097560028, 'mdcr_inpat_margin': 0.21986520273642998, 'mdcr_outpat_margin': 0.06804753492450301, 'mdcr_margin': 0.026983108335128264, 'mdcdinclHMO_inpat_day_s

In [73]:
featureimportance_table.reset_index(drop=True,inplace=True)
predictions_table.reset_index(drop=True,inplace=True)

In [74]:
if (create_models_by_partition == True):
    featureimportance_table.to_csv(Output_results_directory+partitioned_models_directory+featureimportance_table_file,index=False)
    predictions_table.to_csv(Output_results_directory+partitioned_models_directory+predictions_table_file,index=False)
else:
    featureimportance_table.to_csv(Output_results_directory+unpartitioned_models_directory+featureimportance_table_file,index=False)
    predictions_table.to_csv(Output_results_directory+unpartitioned_models_directory+predictions_table_file,index=False)

save_important_variables(lasso_important_variables)

In [75]:
featureimportance_table.head()

,Feature,Importance,Model,QualityStarCluster
0,OPPS_Medicare_Allowed_Amount_STATE_SCORE,0.025344,GBT,1Star
1,OPPS_Medicare_Payment_Amount_STATE_SCORE,0.024304,GBT,1Star
2,OPPS_Total_Submitted_Charges_NATIONAL_SCORE,0.070376,GBT,1Star
3,any_home_office_costs_Y,0.001344,GBT,1Star
4,charity_uncomp_expnsshr_only10,0.036091,GBT,1Star


In [76]:
predictions_table.head(100)

,QualityStarCluster,Medicare_provider_number,Hospital_name,State,"Hospital_Compare_5-star_rating_(October_2018,_NA=Not_Available)",holdout_sample,Class_3,Class_5,Class_6,Class_10,Relative_price_for_outpatient_services,predict,TrainTest,Model
0,1Star,330286,Good Samaritan Hospital,NY,1.0,0,1,2,2,3,191.0,4,train,GBT
1,1Star,30024,St. Josephs Hospital & Medical\nCtr,AZ,2.0,0,2,4,4,7,291.0,4,train,GBT
2,1Star,50616,St. Johns Pleasant Valley\nHospital,CA,2.0,0,3,5,6,9,375.0,5,train,GBT
3,1Star,100029,North Shore Medical Center\nAnd Fmc C,FL,1.0,0,3,5,6,10,563.0,5,train,GBT
4,1Star,450742,Baylor Scott & White - Lake\nPointe,TX,1.0,0,3,5,6,10,546.0,5,train,GBT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1Star,140080,Presence Saint Francis\nHospital,IL,2.0,0,2,3,3,5,232.0,3,train,GBT
96,1Star,140164,Memorial Hospital Of\nCarbondale,IL,1.0,0,2,4,4,7,303.0,4,train,GBT
97,1Star,70031,The Griffin Hospital,CT,1.0,0,1,2,2,3,191.0,2,train,GBT
98,1Star,190008,Terrebonne General Medical\nCenter,LA,2.0,0,2,3,4,6,260.0,3,train,GBT


In [77]:
print(featureimportance_table.shape)
print(predictions_table.shape)

(244, 4)
(2950, 14)


In [78]:
predictions_table[target_col].value_counts(normalize=True) * 100

3    24.881356
4    24.847458
2    20.237288
1    15.220339
5    14.813559
Name: Class_5, dtype: float64